In [1]:
import numpy as np
import random

import math
import time

In [2]:
import torch as th

In [3]:
device = th.device('cuda')

In [4]:
from gene_mat import revers_mat, trans_rev, trans_mat

In [5]:
from dcj_comp import dcj_dist

In [6]:
from gene_mat import gen_data_file, gen_dataset

In [ ]:
n = 200
tmp = np.array([range(1,n + 1)])
mat_list = []
for _ in range(20):
    p1,p2,p3,p4 = [random.randint(0,n) for _ in range(4)]
    mat_list.append(trans_rev(n, p1, p2, p3, p4))
tmp_tensor = th.tensor(tmp, dtype = th.float, device = device)
mat_tensor = th.tensor(np.array(mat_list), dtype = th.float, device = device)
test = th.matmul(tmp_tensor, mat_tensor)

In [ ]:
n = 10
print(time.ctime())
test = th.tensor(np.diag(np.repeat(1,n)), 
                 dtype = th.float, device = device)
mat_op_list = [trans_rev, trans_mat, revers_mat]

print(time.ctime())
for _ in range(10):    
    op_list = np.array([mat_op_list[random.randint(0,2)](
    n, *[random.randint(0,n) for _ in range(4)]) 
           for _ in range(20)])
    mat_tensor = th.tensor(op_list, dtype = th.float, device = device)
    test = th.matmul(test, mat_tensor)
#     print(test[0])
#     print(mat_tensor[0])
#     mat_tensor = mat_tensor[th.randperm(mat_tensor.shape[0])]
print(time.ctime())

In [ ]:
# p1,p2,p3,p4 = [random.randint(0,n) for _ in range(4)]
# for _ in range(2000):
#     op_type = random.randint(0,2)
#     if op_type == 0:
#         op_mat = trans_rev(n, p1, p2, p3, p4)
#     elif op_type == 1:
#         op_mat = trans_mat(n, p1, p2, p3, p4)
#     else:
#         op_mat = revers_mat(n, p1, p2)
#     op_list.append(op_mat)

---

In [ ]:
n, vol = 50, 2000
gene = th.tensor(np.arange(1, n+1).reshape((1,n)), dtype = th.float, device = device)
oriseq = gene.cpu().numpy().astype(np.int32).reshape(n,).tolist()

genome = []
genome.append(oriseq)

t_dist = [0 for _ in range(vol)]

In [ ]:
test = th.tensor(np.diag(np.repeat(1,n)), 
                 dtype = th.float, device = device)
mat_op_list = [trans_rev, trans_mat, revers_mat]
rand_op = [1 for _ in range(vol)] #[random.randint(0,2) for _ in range(vol)]
param_op = [rand_param(n, op_type) for op_type in rand_op] #[[random.randint(0,n) for _ in range(4)] for _ in range(vol)]
t_dist = np.add(t_dist, [1 if x == 2 else 2 for x in rand_op])

# op_list = np.array([mat_op_list[random.randint(0,2)](
# n, *[random.randint(0,n) for _ in range(4)]) 
#        for _ in range(20)])
op_list = np.array([mat_op_list[op](n, *param) 
       for op, param in zip(rand_op, param_op)])


mat_tensor = th.tensor(op_list, dtype = th.float, device = device)
test = th.matmul(test, mat_tensor)
trans_seq = th.matmul(gene, test)

In [ ]:
new_seq = trans_seq.cpu().numpy().astype(np.int32)
for x, d, o, op_param in zip(new_seq, t_dist, rand_op, param_op):
    if dcj_dist(oriseq, x[0])[-1] != d:
        print(x[0], '***\t', dcj_dist(oriseq, x[0])[-1], d, mat_op_list[o].__name__, op_param)
    if x[0].tolist() in genome:
        continue
    genome.append(x[0].tolist())

In [ ]:
def generate_seq_pos(gene):
    if len(gene) == 1:
        return [[gene[0]]]
    res = []
    for i in range(len(gene)):
        g = gene[i]
        rest = gene[0:i] + gene[i+1:]
        tmp = generate_seq_pos(rest)
        res += [[g] + x for x in tmp] #+ [[-g] + x for x in tmp]
    return res

In [ ]:
genome = []

In [ ]:
n, vol = 100, 20000
gene = th.tensor(np.arange(1, n+1).reshape((1,n)), dtype = th.float, device = device)
oriseq = gene.cpu().numpy().astype(np.int32).reshape(n,).tolist()
genome.append(oriseq)

t_dist = [0 for _ in range(vol)]

test = th.tensor(np.diag(np.repeat(1,n)), 
                 dtype = th.float, device = device)

In [ ]:
for _ in range(10):
    mat_op_list = [trans_rev, trans_mat, revers_mat]
    rand_op = [random.randint(0,2) for _ in range(vol)]
    param_op = [rand_param(n, op_type) for op_type in rand_op] #[[random.randint(0,n) for _ in range(4)] for _ in range(vol)]
    t_dist = np.add(t_dist, [1 if x == 2 else 2 for x in rand_op])

    op_list = np.array([mat_op_list[op](n, *param) 
           for op, param in zip(rand_op, param_op)])


    mat_tensor = th.tensor(op_list, dtype = th.float, device = device)
    test = th.matmul(test, mat_tensor)
    trans_seq = th.matmul(gene, test)
    new_seq = trans_seq.cpu().numpy().astype(np.int32)

    # tmp = np.unique(new_seq, axis=0).reshape((-1,n))
    # tmp.shape
    count = 0
    tmp = []
    for x, d, o, op_param in zip(new_seq, t_dist, rand_op, param_op):
        if dcj_dist(oriseq, x[0])[-1] != d:
    #         print(x[0], '***\t', dcj_dist(oriseq, x[0])[-1], d, mat_op_list[o].__name__, op_param)
            count += 1
            continue
        tmp.append(x[0])

    tmp = np.unique(tmp, axis = 0)
    if len(tmp) != 0:
        genome += tmp.tolist()
#     print(count)

    genome = np.unique(genome, axis = 0).tolist()

In [14]:
gen_data_file(10, 1000, 5, 'data/data_10.mat')

In [11]:
gen_data_file(50, 1000, 20, 'data/data_50.mat')

In [12]:
gen_data_file(100, 1000, 40, 'data/data_100.mat')

In [13]:
gen_data_file(500, 1000, 200, 'data/data_500.mat')

In [10]:
# gen_data_file(2000, 1, 800, 'data_2000.mat')

In [1]:
import torch as th
from torch.nn import Linear as linear

In [2]:
from torch.nn import Module
from gene_mat import gen_op_mat

In [3]:
class trans_linear(linear):
    def __init__(self, l: int,
                 device=None, dtype=None) -> None:
        super(trans_linear, self).__init__(in_features = l, 
                                           out_features = l, 
                                           bias = False, 
                                           device = device, 
                                           dtype = dtype)
        self.weight = th.nn.Parameter(th.tensor(gen_op_mat(l, 1)[0][0], 
                                                dtype = th.float, device = device))

In [4]:
def t_l(x, step):
    for _ in range(step):
        x = trans_linear(x.shape[-1], device = 'cuda')(x)
    return x

In [5]:
l,n,step = 1000, 500, 500
x = th.zeros(n, l, dtype = th.float, device = 'cuda')
for i in range(x.shape[0]):
    x[i] = th.randperm(l) + 1

In [6]:
x

tensor([[966., 281., 611.,  ...,  47., 535., 151.],
        [ 56., 307., 399.,  ..., 488., 552., 266.],
        [929., 363., 502.,  ..., 157., 331., 193.],
        ...,
        [532., 962., 764.,  ..., 757., 839.,   9.],
        [854., 768., 730.,  ..., 106., 740., 591.],
        [133., 536., 271.,  ..., 414., 469., 522.]], device='cuda:0')

In [7]:
t_l(x, step)

tensor([[ 877.,  710.,  973.,  ...,  622.,  356., -507.],
        [ 495.,  473.,  761.,  ...,  254.,  576., -726.],
        [ 313.,  795.,  778.,  ...,  172.,  899., -901.],
        ...,
        [ 624.,  645.,  403.,  ...,  796.,  287., -465.],
        [ 149.,   33.,  954.,  ...,  630.,   20., -686.],
        [ 142.,  498., 1000.,  ...,  754.,  589., -823.]], device='cuda:0',
       grad_fn=<MmBackward0>)